In [2]:
#from __future__ import division # If in Python 2
import numpy as np
import pandas as pd
import h5py
import matplotlib
%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
print(plt.get_backend())

import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick

from cycler import cycler
import sys, os, glob
import pytz
import lmfit
import time

from scipy.signal import *
from scipy.stats import *
from scipy.interpolate import *
from scipy.integrate import odeint

# use latex fonts/mathematica color scheme
#matplotlib.rc('text', usetex=True)
#matplotlib.rc('font', family='serif')
#matplotlib.verbose.level = 'debug-annoying'
#matplotlib.rcParams['mathtext.fontset'] = 'cm'

matplotlib.rcParams.get('font.sans-serif').insert(0,u'Myriad Web Pro')
matplotlib.rcParams.get('font.serif').remove(u'Times')
matplotlib.rcParams.get('font.serif').insert(0,u'Times')

plotparams = {'legend.fontsize'      : 10,
              'figure.figsize'       : (3.25, 2.75),
              'figure.dpi'           : 300,
              'axes.labelsize'       : 7,
              'axes.titlesize'       : 7,
              'xtick.labelsize'      : 7,
              'ytick.labelsize'      : 7,
              'figure.subplot.bottom': 0.1,
              'figure.subplot.hspace': 0.1,
              'figure.subplot.left'  : 0.125,
              'figure.subplot.right' : 0.95,
              'figure.subplot.top'   : 0.95,
              'figure.subplot.wspace': 0.1,
              'mathtext.fontset'     : 'custom',
              'mathtext.rm'          : matplotlib.rcParams.get('font.sans-serif')[0],
              'mathtext.it'          : matplotlib.rcParams.get('font.sans-serif')[0] + u':italic',
              'mathtext.bf'          : matplotlib.rcParams.get('font.sans-serif')[0] + u':bold',
              'mathtext.sf'          : matplotlib.rcParams.get('font.sans-serif')[0]}

matplotlib.rcParams.update(plotparams)

# cyclers for colors and line styles
line_cycle = cycler('linestyle', ["-","-.","--"])

#color_cycle_trimmed = cycler('color', ["#62ad64", "#9750a1", "#b8893a", "#6778d0", "#b8475f"])
color_cycle = cycler('color', ["#62ad64", "#9750a1", "#b8893a", "#6778d0", "#b8475f"])
clrs = ["#62ad64", "#9750a1", "#b8893a", "#6778d0", "#b8475f"]
clrs2 = ["#e18e9c", "#53c0da", "#c0af6f", "#b5a0dd", "#71c19b"]
clrs3 = ["#900094", "#c799ff", "#002ecd"]

nbAgg


In [7]:
data = np.array([[1, 1000],[4.4, 3.9],[0.018, 0.013]]).transpose()
df = pd.DataFrame(data, columns = ['freq','Er','tanD'])
df

,freq,Er,tanD
0,1.0,4.4,0.018
1,1000.0,3.9,0.013


In [ ]:
def RPerm(x, e_s, e_inf, tau):
    return (e_inf + (e_s - e_inf)/(1 + (2*np.pi*x*tau)**2))
def TanD(x, e_s, e_inf, tau):
    return ((e_s - e_inf)*2*np.pi*x*tau/(e_s + e_inf*(2*np.pi*x*tau)**2))
def Debye_fit(params, x, Er = None, TanD = None):
    